In [ ]:
myPackages <- c("dplyr", "tidyr", "purrr")
lapply(myPackages, library, character.only = TRUE)
library(sjlabelled, lib.loc = "~/rlib")

In [3]:
gallupRaw <- rio::import("~/gallup data/Gallup_World_Poll_021723.rds")
dim(gallupRaw)

[1] 2591054    2694

In [4]:
gallupExt <- gallupRaw

In [5]:
gallupQuestions <- rio::import("~/gallup data/GallupQuestions051523.csv")[-3]
glimpse(gallupQuestions)

Rows: 2,691
Columns: 2
$ QTag            <chr> "COUNTRY_ISO2", "COUNTRY_ISO3", "COUNTRYNEW", "DEGURBA…
$ `Question Text` <chr> "Country ISO alpha-2 code", "Country ISO alpha-3 code"…


## DK/Refused/NA/No Answer

In [6]:
dk.text <- c("(DK(\\b|\\W))|(((Do\\Wnot)|dont|don\\?t)\\Wknow\\W{0,1}$)")
refused.text <- c("(refuse)|(RF(\\b|\\W))")
na.text <- c("(\\(does\\Wnot\\Wapply)|(Not\\WApplicable)|(^NA$)|(\\(N\\W{0,1}A\\))|(No\\Wanswer)|(No\\Wresponse)|(Not\\WAvailable\\W{0,1}$)") 
#note that Gallup_World_Poll_021723.sav's WP12 labels is missing "Not Available" (97)
texttoNA <- paste(dk.text, refused.text, na.text, sep = "|")

In [7]:
lapply(colnames(gallupExt), 
                       function(var){
    if (sum(grepl(texttoNA, get_labels(gallupExt[, var]), ignore.case = T)) > 0){
        NAdetails <- data.frame(matrix(
            c(var,
              get_label(gallupExt[, var]),
              get_labels(gallupExt[, var])[grepl(texttoNA, get_labels(gallupExt[, var]), ignore.case = T)]),
            1))
        colnames(NAdetails)[1:2] <- c("QTag", "Short Text")
        return(NAdetails)
        }
}) %>% bind_rows %>% distinct(X3, X4, X5, .keep_all = T)

QTag,Short Text,X3,X4,X5
<chr>,<chr>,<chr>,<chr>,<chr>
WP5889,Questionnaire Serial Number,(DK),(Refused),NA
COUNTRY_ISO2,Country ISO alpha-2 code,DK,NA,NA
EMP_WORK_HOURS,Total Number of Hours Work Per Week,No answer,NA,NA
REGION_DEU,Region Germany,Dont know,Refused,NA
REGION_GBR,Region United Kingdom,(Dont know),(Refused),NA
REGION_GHA,Region Ghana,(DK),(REFUSED),NA
WP10004,Age Started Smoking,(Refused),NA,NA
WP10206,Hiring or Letting Go,(Does not apply),(DK),(Refused)
WP10633,Receiving Pension,(DK),(Refused),(Not Applicable)


In [8]:
Gallup.NA <- colnames(gallupExt) %>% 
    lapply(function(var){ 
        labels <- get_labels(gallupExt[, var])
        values <- get_values(gallupExt[, var])
        dk.value <- values[grepl(dk.text, labels, ignore.case = T)]
        refused.value <- values[grepl(refused.text, labels, ignore.case = T)]
        na.value <- values[grepl(na.text, labels, ignore.case = T)]

        if (sum(grepl(texttoNA, labels, ignore.case = T)) > 0 &
            ! var %in% "COUNTRY_ISO2"){
            return(c(QTag = var,
                     "Short Text" = get_label(gallupExt[, var]),
                     "(dk).value" = dk.value,
                     "(refused).value" = refused.value,
                     "(na).value" = na.value,
                     "Response Options" = paste(labels, collapse = ", "),
                     "Response Values" = paste(values, collapse = ", ")))
                               }}) %>% 
    bind_rows %>%
    select(QTag, `Short Text`, `(dk).value`, `(refused).value`, `(na).value`, `Response Options`, `Response Values`)

nrow(Gallup.NA)
Gallup.NA %>% distinct(`(dk).value`, `(refused).value`, `(na).value`, .keep_all = T)

[1] 2464

QTag,Short Text,(dk).value,(refused).value,(na).value,Response Options,Response Values
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
WP5889,Questionnaire Serial Number,9998,9999,NA,", 9999, (DK), (Refused)","1, 9997, 9998, 9999"
WP5,Country,998,999,NA,"Search code list, United States, Egypt, Morocco, Lebanon, Saudi Arabia, Jordan, Syria, Turkey, Pakistan, Indonesia, Bangladesh, United Kingdom, France, Germany, Netherlands, Belgium, Spain, Italy, Poland, Hungary, Czech Republic, Romania, Sweden, Greece, Denmark, Iran, Hong Kong, Singapore, Japan, China, India, Venezuela, Brazil, Mexico, Nigeria, Kenya, Tanzania, Israel, Palestinian Territories, Ghana, Uganda, Benin, Madagascar, Malawi, South Africa, Canada, Australia, Philippines, Sri Lanka, Vietnam, Thailand, Cambodia, Laos, Myanmar, New Zealand, Angola, Botswana, HOLD, HOLD, Ethiopia, Mali, Mauritania, Mozambique, Niger, Rwanda, Senegal, Zambia, South Korea, Taiwan, Afghanistan, Belarus, Georgia, Kazakhstan, Kyrgyzstan, Moldova, Russia, Ukraine, Burkina Faso, Cameroon, Sierra Leone, Zimbabwe, Costa Rica, Albania, Algeria, Andorra, Antigua & Barbuda, Argentina, Armenia, Austria, Azerbaijan, Bahamas, Bahrain, Barbados, Belize, Bhutan, Bolivia, Bosnia and Herzegovina, Brunei, Bulgaria, Burundi, Cape Verde, Central African Republic, Chad, Chile, Colombia, Comoros, Congo (Kinshasa), Congo Brazzaville, Croatia, Cuba, Cyprus, Djibouti, Dominica, Dominican Republic, Ecuador, El Salvador, Equatorial Guinea, Eritrea, Estonia, Fiji, Finland, Gabon, Grenada, Guatemala, Guinea, Guinea-Bissau, Guyana, Haiti, Honduras, Iceland, Iraq, Ireland, Island Nations (11), Ivory Coast, Jamaica, Kiribati, Kuwait, Latvia, Lesotho, Liberia, Libya, Lichtenstein, Lithuania, Luxembourg, North Macedonia, Malaysia, Maldives, Malta, Marshall Islands, Mauritius, Micronesia, Monaco, Mongolia, Montenegro, Namibia, Nauru, Nepal, Nicaragua, North Korea, Norway, Oman, Palau, Panama, Paraguay, Peru, Portugal, Puerto Rico, Qatar, Saint Lucia, Samoa, San Marino, Sao Tome & Principe, Serbia, Seychelles, Slovakia, Slovenia, Solomon Islands, Somalia, St. Kitts & Nevis, St. Vincent & Grenadines, Sudan, Suriname, Eswatini, Switzerland, Tajikistan, The Gambia, Togo, Tonga, Trinidad & Tobago, Tunisia, Turkmenistan, Tuvalu, United Arab Emirates, Uruguay, Uzbekistan, Vanuatu, Yemen, Kosovo, Somaliland region, US Hispanic, Timor Leste, Northern Cyprus, Nagorno-Karabakh Region, Papua New Guinea, South Sudan, Macau, Reunion Island, African Country, Arab Country, Other Islamic Country, Other Country, None, (None), (DK), (Refused)","0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 900, 901, 902, 903, 907, 997, 998, 999"
EMP_WORK_HOURS,Total Number of Hours Work Per Week,NA,NA,98,"Less than 15 hours per week, 15 to 29 hours per week, 30 to 39 hours per week, 40 to 49 hours per week, 50 or more hours per week, No answer","1, 2, 3, 4, 5, 98"
M1,Not Enough Money: Healthcare,3,4,NA,"Yes, No, (DK), (Refused)","1, 2, 3, 4"
M30,Current Economic Conditions,8,9,NA,"Poor, Only fair, Good, Excellent, (DK), (Refused)","1, 2, 3, 4, 8, 9"
REGION_ALB,Region Albania,98,99,NA,"Berat, Bulqi

In [9]:
#2023 June 20th, WP12 has the option 97: Not Available but not shown in the label of original sav file, so we add it here
Gallup.NA[Gallup.NA$QTag %in% "WP12", "(na).value"] <- "97"
Gallup.NA[Gallup.NA$QTag %in% "WP12",]

QTag,Short Text,(dk).value,(refused).value,(na).value,Response Options,Response Values
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
WP12,Residents 15+ in Household,98,99,97,", , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , 96+, (DK), (Refused)","1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 98, 99"


In [10]:
Gallup.label.list <- colnames(gallupExt) %>%
    lapply(function(var){
        labels <- get_labels(gallupExt[, var])
        values <- get_values(gallupExt[, var])
        return(list(labels = labels, values = values))
    })
names(Gallup.label.list) <- colnames(gallupExt)

In [11]:
lapply(Gallup.label.list[!names(Gallup.label.list) %in% Gallup.NA$QTag], function(x){
    label = x$labels[length(x$labels)]
    value = x$values[length(x$values)]
                     
    if(is.null(value) & !is.null(label)){
        return(as.data.frame(cbind(label, value = NA)))
    } else {
        return(as.data.frame(cbind(label, value)))
    }
}) %>% compact %>% bind_rows(.id = "QTag") %>% filter(!grepl("REGION", QTag)) %>% distinct(label, .keep_all = T)

QTag,label,value
<chr>,<chr>,<chr>
COUNTRY_ISO2,MV,NA
COUNTRY_ISO3,MDV,NA
COUNTRYNEW,Maldives,NA
EMP_2010,Out of workforce,6
EMP_FTEMP,Full-time for an employer,1
EMP_LFPR,In Labor Force,1
EMP_UNDER,Underemployed,1
EMP_UNEMP,Unemployed,1
INCOME_5,Richest 20%,5


In [12]:
Gallup.NA %>% filter((is.na(`(dk).value`) & !is.na(`(refused).value`)) |
         (!is.na(`(dk).value`) & is.na(`(refused).value`)))

QTag,Short Text,(dk).value,(refused).value,(na).value,Response Options,Response Values
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
WP10004,Age Started Smoking,NA,100,NA,"99+, (Refused)","99, 100"
WP10726,Muslim a Good Swiss Patriot,9,NA,NA,"Yes, No, (DK/NA)","1, 2, 9"
WP10727,Religious Freedom for Swiss Identity,9,NA,NA,"Not important at all, , , , Very important, (DK/NA)","1, 2, 3, 4, 5, 9"
WP10728,Swiss Muslims Discriminated Against,9,NA,NA,"Yes, No, (DK/NA)","1, 2, 9"
WP10729,Contradiction Between Democracy and Islam,9,NA,NA,"Rather agree, Rather disagree, DK/NA","1, 2, 9"
WP10733,Harmed Swiss Reputation: Referendum on Minarets,9,NA,NA,"Did not harm at all, , , , Harmed a lot, (DK/NA)","1, 2, 3, 4, 5, 9"
WP10932,First Try Smoking,NA,100,NA,"99+, (Refused)","99, 100"
WP10933,First Occasionally Smoke,NA,100,NA,"99+, (Refused)","99, 100"
WP10950,Age 1st Household Member,NA,100,NA,", , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , 99+, (Refused)","1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100"


In [13]:
Gallup.NA %>%
  filter(is.na(`(dk).value`) & is.na(`(refused).value`) & !is.na(`(na).value`))

QTag,Short Text,(dk).value,(refused).value,(na).value,Response Options,Response Values
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
EMP_WORK_HOURS,Total Number of Hours Work Per Week,NA,NA,98,"Less than 15 hours per week, 15 to 29 hours per week, 30 to 39 hours per week, 40 to 49 hours per week, 50 or more hours per week, No answer","1, 2, 3, 4, 5, 98"


In [14]:
QTag_DKRefused <- Gallup.NA %>% filter((!is.na(`(dk).value`) & !is.na(`(refused).value`))) %>% pull(QTag)

lapply(Gallup.label.list[names(Gallup.label.list) %in% QTag_DKRefused], function(x){
    
    labels <- x$labels[!grepl(texttoNA, x$labels, ignore.case = T)]
    values <- x$values[!grepl(texttoNA, x$labels, ignore.case = T)]
    
    valuesNA <- x$values[grepl(texttoNA, x$labels, ignore.case = T)][1]
    
    label <- labels[length(labels)]
    value <- values[length(values)]
    
    prevalue <- values[length(values)-1]
                     
    if(length(labels) > 2 && (value-prevalue != 1)){
        return(as.data.frame(cbind(label, value)))
        }
    
}) %>% compact %>% bind_rows(.id = "QTag") %>% filter(!grepl("REGION", QTag)) %>% distinct(label, .keep_all = T)

QTag,label,value
<chr>,<chr>,<chr>
WP5,(None),997
WP10005,Other (specify) ______________________________________________,7
WP10113,"Other, specify: _______________________________",97
WP10498,Other,97
WP1163,Oil,11
WP12211,None of these,8
WP1223,Domestic partner,8
WP1226,Other (list),11
WP12459,Other (specify) _________,97


In [15]:
str(gallupExt$WP8872)
get_labels(gallupExt$WP8872)

 num [1:2591054] NA NA NA NA NA NA NA NA NA NA ...
 - attr(*, "label")= chr "Heat Home"
 - attr(*, "format.spss")= chr "F8.0"
 - attr(*, "labels")= Named num [1:18] 1 2 3 4 5 6 7 8 9 10 ...
  ..- attr(*, "names")= chr [1:18] "Coal" "Oil" "Propane or natural gas" "Wood" ...


[1] "Coal"                                                         
 [2] "Oil"                                                          
 [3] "Propane or natural gas"                                       
 [4] "Wood"                                                         
 [5] "Electricity"                                                  
 [6] "Peat"                                                         
 [7] "Paraffin"                                                     
 [8] "Charcoal"                                                     
 [9] "Dung"                                                         
[10] "Straw/Firewood"                                               
[11] "Solar"                                                        
[12] "Wind"                                                         
[13] "Central heat from a distance / District heating / energy grid"
[14] "Grass"                                                        
[15] "Other (Do not list)"                                          
[16] "(NA) (I dont heat my home)"                                   
[17] "(DK)"                                                         
[18] "(Refused)"

In [16]:
str(gallupExt$WP12211)
get_labels(gallupExt$WP12211)

 num [1:2591054] NA NA NA NA NA NA NA NA NA NA ...
 - attr(*, "label")= chr "Biggest Issue Facing Family"
 - attr(*, "format.spss")= chr "F8.0"
 - attr(*, "labels")= Named num [1:9] 1 2 3 4 5 6 8 98 99
  ..- attr(*, "names")= chr [1:9] "Lack of food" "Lack of job/Unemployment" "Price increase of food products" "Lack of money/Poverty" ...


[1] "Lack of food"                    "Lack of job/Unemployment"       
[3] "Price increase of food products" "Lack of money/Poverty"          
[5] "Flooding/Heavy rain"             "Drought/Lack of rain"           
[7] "None of these"                   "(DK)"                           
[9] "(Refused)"

# non-zero NONE

Some questions has the option `None` with non-zero value. They are not neccessary to be NA but we have to be caution

In [17]:
none.text <-c("(none)")

In [18]:
lapply(colnames(gallupExt), 
                       function(var){
                           
    values <- get_values(gallupExt[, var])                       
    labels <- get_labels(gallupExt[, var])
    none.values <- values[grepl(none.text, labels, ignore.case = T)]
    none.value <- none.values[none.values != 0]  
    none.label <- labels[values %in% none.value]
                           
    
        NAdetails <- data.frame(matrix(
            c(var,
              get_label(gallupExt[, var]),
              none.value,
              none.label),
            1))
        colnames(NAdetails)[1:2] <- c("QTag", "Short Text")
        return(NAdetails)
        
}) %>% bind_rows %>% distinct(X3, X4, X5, X6, .keep_all = T)

QTag,Short Text,X3,X4,X5,X6
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
WP5889,Questionnaire Serial Number,NA,NA,NA,NA
WP5,Country,907,997,None,(None)
WP10007,Quit Smoking Aids,7,(None of these),NA,NA
WP10067,Knowledge About Muslim Beliefs,4,None at all,NA,NA
WP10936,Preferred Stop Smoking Aid,96,(None of these),NA,NA
WP11523,Power Cuts Household,9,None – There were not power cuts,NA,NA
WP11524,Power Cuts Community,9,None - There were not power cuts,NA,NA
WP1162,Sanitation/Toilet,12,(None),NA,NA
WP12211,Biggest Issue Facing Family,8,None of these,NA,NA


In [19]:
Gallup.none <- lapply(colnames(gallupExt), 
                       function(var){ 
    values <- get_values(gallupExt[, var])                       
    labels <- get_labels(gallupExt[, var])
    none.values <- values[grepl(none.text, labels, ignore.case = T)]
    none.value <- none.values[none.values != 0]  
    none.label <- labels[values %in% none.value]
                        
    if (length(none.value) > 0){
        return(c(QTag = var,
                 "Short Text" = get_label(gallupExt[, var]),
                 "non-zero.none.value" = paste(none.value, collapse = ", "),
                 "Response Options" = paste(labels, collapse = ", "),
                 "Response Values" = paste(get_values(gallupExt[, var]), collapse = ", ")))
                           }}) %>% bind_rows 
nrow(Gallup.none)
Gallup.none %>% distinct(`non-zero.none.value`, .keep_all = T)

[1] 78

QTag         Short Text                            non-zero.none.value
1  WP5          Country                               907, 997           
2  WP10007      Quit Smoking Aids                     7                  
3  WP10067      Knowledge About Muslim Beliefs        4                  
4  WP10936      Preferred Stop Smoking Aid            96                 
5  WP11523      Power Cuts Household                  9                  
6  WP1162       Sanitation/Toilet                     12                 
7  WP12211      Biggest Issue Facing Family           8                  
8  WP1233       Religion                              26                 
9  WP13125_2013 2013 Leader Codes                     907                
10 WP14677      Increasing Job Opportunities in Egypt 5                  
11 WP5034       News about Palestine                  3                  
12 WP7022       Main Source of Health Information     97                 
13 WP7470       Relations with Other CIS Countries    997                
14 WP891        News in My Country                    98                 
15 WP9051       Borrow Money                          1                  
16 WP9917       Vote for President                    995                
   Response Options                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

# Binary Responses

In [20]:
Gallup.Binary <- lapply(colnames(gallupExt), 
                       function(var){
    labels <- get_labels(gallupExt[, var])
    values <- get_values(gallupExt[, var])
    nonna.values <- as.numeric(values[!grepl(texttoNA, labels, ignore.case = T)]) %>% sort
                           
    if (identical(nonna.values, c(1, 2)))
        return(c(QTag = var,
                 "Short Text" = get_label(gallupExt[, var]),
                 "Response Options" = paste(labels, collapse = ", "),
                 "Response Values" = paste(values, collapse = ", ")))
}) %>% bind_rows %>% merge(gallupQuestions, by = "QTag")

nrow(Gallup.Binary)
Gallup.Binary %>% group_by(`Response Values`) %>% {merge(slice(., 1), summarize(., `n of QTag with the corresponding Response Values` = n()))}

[1] 1377

Response Values,QTag,Short Text,Response Options,Question Text,n of QTag with the corresponding Response Values
<chr>,<chr>,<chr>,<chr>,<chr>,<int>
"1, 2",REGION2_NZL,Region 2 New Zealand,"North Island, South Island",Region 2 New Zealand,24
"1, 2, 3",WP3476,Have Passport,"Yes, No, Refused","Do you, yourself, have a passport, or not?",1
"1, 2, 3, 4",M1,Not Enough Money: Healthcare,"Yes, No, (DK), (Refused)",Have there been times in the past 12 months when you did not have enough money to pay for healthcare and/or medicines that you or your family needed?,1249
"1, 2, 3, 4, 5",WP10633,Receiving Pension,"Yes, No, (DK), (Refused), (Not Applicable)",Are you receiving a pension or some form of financial remuneration for work you did while you were employed? (asked only of those who are age 61 or older),23
"1, 2, 3, 8, 9",WP21758,Temporarily Stopped Working at Job or Business as a Result of Coronavirus Situation,"Yes, No, (Does not apply/No job), (DK), (Refused)",Have you experienced each of the following as a result of the (local term for coronavirus) situation? Temporarily stopped working at your job or business,5
"1, 2, 4, 5",WP4941,Efforts to Fight Terrorism,"Yes, doing enough, No, not doing enough, (DK), (Refused)","Do you think the government is doing enough to fight terrorism, or not?",2
"1, 2, 7, 8, 9",WP11533,Go to Bed Feeling Hungry,"Agree, Disagree, (Not applicable), (DK), (Refused)","Do you agree or disagree with the following statements? These days, I often go to bed feeling hungry.",7
"1, 2, 8",WP3570,Foreign Currency Account,"Yes, No, (DK)",Do you or any member of your family in this household have a bank account in foreign currency?,1
"1, 2, 8, 9",WP10120,Consider Self IDP,"Yes, No, (DK), (Refused)",Do you consider yourself to be an IDP (internally displaced person)? (asked only of those who have moved),48
